In [1]:
import os
import pandas as pd

# Función para extraer el nombre del modelo del nombre del archivo
def extract_model_name(filename):
    parts = filename.split('_')
    for part in parts:
        if part.startswith(('Llama', 'Yi', 'gemma', 'Mistral', 'ChatGPT', 'phi')):
            return part
    return 'Unknown'

# Función para analizar los fallos en las preguntas y verificar en en_l0
def analyze_failures_with_en_l0(folder):
    question_failures = {}  # Diccionario para almacenar los fallos por pregunta
    model_columns = set()  # Conjunto para almacenar los nombres de los modelos (columnas dinámicas)
    
    for filename in os.listdir(folder):
        # Filtrar solo los archivos que comienzan con 'paraphrases' y no terminan en '_copia'
        if filename.startswith('paraphrases') and not filename.endswith('_copia.xlsx'):
            filepath = os.path.join(folder, filename)
            model_name = extract_model_name(filename)  # Extraer el nombre del modelo
            model_columns.add(model_name)  # Agregar el modelo a las columnas dinámicas
            
            try:
                # Leer las hojas en_l1 y en_l0
                df_en_l1 = pd.read_excel(filepath, sheet_name='en_l1', engine='openpyxl')
                df_en_l0 = pd.read_excel(filepath, sheet_name='en_l0', engine='openpyxl')
                
                # Recorrer las filas del DataFrame en_l1
                for index, row in df_en_l1.iterrows():
                    question_id = index + 2  # Ajustar el índice para que coincida con la fila en Excel
                    question_text = row['question']
                    
                    # Inicializar la pregunta en el diccionario si no existe
                    if question_id not in question_failures:
                        question_failures[question_id] = {'question': question_text, 'failures': 0}
                    
                    # Verificar si la pregunta fue fallada en en_l1
                    if row['llm_answer_filtered'] != row['correct_answer']:
                        question_failures[question_id]['failures'] += 1
                    
                    # Verificar si la pregunta fue acertada o fallada en en_l0
                    en_l0_row = df_en_l0.iloc[index]  # Obtener la fila correspondiente en en_l0
                    if en_l0_row['llm_answer_filtered'] == en_l0_row['correct_answer']:
                        question_failures[question_id][model_name] = 'Acertada'
                    else:
                        question_failures[question_id][model_name] = 'Fallada'
            
            except Exception as e:
                print(f"Error procesando el archivo {filename}: {e}")
    
    # Asegurar que todas las preguntas tengan columnas para todos los modelos
    for question_id in question_failures:
        for model in model_columns:
            if model not in question_failures[question_id]:
                question_failures[question_id][model] = 'No evaluada'
    
    return question_failures

# Función para guardar los resultados en un archivo Excel
def save_failures_to_excel(failures, output_excel):
    # Convertir el diccionario a un DataFrame
    failures_df = pd.DataFrame.from_dict(failures, orient='index')
    failures_df.index.name = 'id'
    failures_df.reset_index(inplace=True)
    
    # Crear una columna auxiliar para verificar si todos los modelos tienen "Acertada"
    model_columns = [col for col in failures_df.columns if col not in ['id', 'question', 'failures']]
    failures_df['all_correct'] = failures_df[model_columns].apply(lambda row: all(val == 'Acertada' for val in row), axis=1)
    
    # Ordenar primero por número de fallos (descendente) y luego por "all_correct" (True primero)
    failures_df = failures_df.sort_values(by=['failures', 'all_correct'], ascending=[False, False])
    
    # Eliminar la columna auxiliar antes de guardar
    failures_df.drop(columns=['all_correct'], inplace=True)
    
    # Guardar el resultado en un archivo Excel
    failures_df.to_excel(output_excel, index=False, engine='openpyxl')
    print(f"Archivo generado: {output_excel}")

# Ejemplo de uso
folder = 'MMLU'  # Carpeta con los archivos Excel
output_excel = 'preguntas_falladas_mmlu.xlsx'

# Ejecutar el análisis
failures = analyze_failures_with_en_l0(folder)
save_failures_to_excel(failures, output_excel)

Archivo generado: preguntas_falladas_mmlu.xlsx
